<a href="https://colab.research.google.com/github/alex-bolshunov/Data_mining_project/blob/main/data_mining_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [432]:
#columns: npi, gender, state, dredentials, speciality, opiods, non opiods, risk skale (overdose / population) ?
#https://www.kaggle.com/datasets/apryor6/us-opiate-prescriptions/data?select=overdoses.csv

In [433]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [434]:
#get data
!curl "https://raw.githubusercontent.com/alex-bolshunov/Data_mining_project/main/prescriber_info.csv" -o prescriber.csv
!curl "https://raw.githubusercontent.com/alex-bolshunov/Data_mining_project/main/overdoses.csv" -o overdose.csv
!curl "https://raw.githubusercontent.com/alex-bolshunov/datasets/refs/heads/main/type_of_meds.csv" -o type_of_meds.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.7M  100 13.7M    0     0  19.8M      0 --:--:-- --:--:-- --:--:-- 19.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1789  100  1789    0     0   6901      0 --:--:-- --:--:-- --:--:--  6907
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4159  100  4159    0     0  15447      0 --:--:-- --:--:-- --:--:-- 15403


In [435]:
#type of meds
meds_types = pd.read_csv("type_of_meds.csv")

In [436]:
#sorting by type
meds_types.sort_values("Type", inplace=True)

#setting new index
new_index = list(range(len(meds_types)))
meds_types['index'] = new_index
meds_types.set_index('index', inplace = True)

In [437]:
#get indecies of rows for each group
meds_indicies = meds_types.loc[:, "Type"].value_counts().cumsum()

In [438]:
#overdose df
overdose_df = pd.read_csv("overdose.csv")

In [439]:
overdose_column_names = ["state_full", "pop", "od", "state"]
overdose_df_types = {
    overdose_column_names[1]: "int32",
    overdose_column_names[2]: "int32",
}

In [440]:
#renaming columns
overdose_df.columns = overdose_column_names

In [441]:
#removing comas
num_columns_to_clean = ["pop", "od"]
overdose_df[num_columns_to_clean] = overdose_df[num_columns_to_clean].map(lambda s: s.replace(",", ""))

In [442]:
#changing datatype
overdose_df = overdose_df.astype(overdose_df_types)

In [443]:
#subset three columns, remove full state name column
overdose_df = overdose_df.loc[:, overdose_column_names[1:]]

In [444]:
#create new variable overdose ratio (overdose / populaiton)
overdose_df["od.ratio"] =  (overdose_df["od"]/overdose_df["pop"]*100)

In [445]:
#summary statistics
overdose_df["od.ratio"].agg(["min", "mean", "median", "max"])

,od.ratio
min,0.005944
mean,0.015944
median,0.014906
max,0.033813


In [446]:
#define the group based on the number
def categorize(num):
  available_groups = "ABC"
  selected_group = ""

  if(num < 0.01): selected_group = available_groups[2]
  elif(num < 0.02): selected_group = available_groups[1]
  else: selected_group = available_groups[0]

  return selected_group

In [447]:
#create variabe od group
overdose_df["od.group"] = overdose_df["od.ratio"].apply(categorize)

In [448]:
#check value counts
overdose_df["od.group"].value_counts().sort_index()

,count
od.group,
A,10
B,34
C,6


In [449]:
overdose_df.iloc[25:30]

,pop,od,state,od.ratio,od.group
25,1015165,125,MT,0.012313,B
26,1868516,125,NE,0.006690,C
27,2790136,545,NV,0.019533,B
28,1323459,334,NH,0.025237,A
29,8899339,1253,NJ,0.014080,B


In [450]:
#testing values
groups = "ABC"
groups_states_dict = {}

#populate dict state:group pairs
for char in groups:
  groups_states_dict.update({
     key:char for key in overdose_df[overdose_df["od.group"] == char]["state"].to_list()
  })

#check if the state in correct group
def check_group(state, group):
  if(group != groups_states_dict.get(state, "")): raise Exception("Group doesn't match")

#loop over values check each pair
def test1(df):
  for state, group in list(df[['state', "od.group"]].itertuples(index=False, name=None)): check_group(state, group)
  print("Test completed")


In [451]:
#prescriber df
prescriber_df = pd.read_csv("prescriber.csv")

p_df = prescriber_df.copy()

In [452]:
prescriber_df.head()

,NPI,Gender,State,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1245278100,F,AL,MD,General Surgery,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35,1
2,1427182161,F,NY,M.D.,General Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
3,1669567541,M,AZ,MD,Internal Medicine,0,43,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1679650949,M,NV,M.D.,Hematology/Oncology,0,0,0,0,0,...,0,0,0,0,17,28,0,0,0,1


In [453]:
#get list of meds
#list_of_meds = list(prescriber_df.columns)[5:255]

In [454]:
#reordering columns accoring to each group
p_cols = prescriber_df.columns.to_list()
p_cols_sorted = list(meds_types.loc[:, "Name"].values)
prescriber_df.columns = p_cols[:5] + p_cols_sorted + p_cols[len(p_cols) - 1:]

In [455]:
#using indecies of sorted col summarize them
#get start and end column names, define new column sum, update index
j = 0
opioid_col_names = ["not.opioid", "opioid", "controlled"]

for i in range(len(meds_indicies)):
  start_col, stop_col = p_cols_sorted[j], p_cols_sorted[meds_indicies[i]-1]
  prescriber_df[opioid_col_names[i]] = prescriber_df.loc[:, start_col: stop_col].apply("sum", axis=1)
  print(start_col, stop_col)
  j = meds_indicies[i]

ABILIFY DOXAZOSIN.MESYLATE
OXYCONTIN DIAZEPAM
AMPHETAMINE.SALT.COMBO ZOLPIDEM.TARTRATE


In [456]:
#subset values, change names of the columns
prescriber_df_select_cols = ["NPI", "Gender", "State", "Credentials", "Specialty", "Opioid.Prescriber"] + opioid_col_names
prescribeer_df_new_cols = pd.Series(prescriber_df_select_cols).apply(lambda s: s.lower()).to_list()
prescriber_df = prescriber_df[prescriber_df_select_cols].copy()
prescriber_df.columns = prescribeer_df_new_cols

In [457]:
prescriber_df.head()

,npi,gender,state,credentials,specialty,opioid.prescriber,not.opioid,opioid,controlled
0,1710982582,M,TX,DDS,Dentist,1,29,0,0
1,1245278100,F,AL,MD,General Surgery,1,1304,79,35
2,1427182161,F,NY,M.D.,General Practice,0,205,59,25
3,1669567541,M,AZ,MD,Internal Medicine,1,1651,106,0
4,1679650949,M,NV,M.D.,Hematology/Oncology,1,134,0,45


In [458]:
#merge 2 dataframes
df = pd.merge(prescriber_df, overdose_df, on = "state")

#check value counts for overdose groups
df["od.group"].value_counts().sort_index()

,count
od.group,
A,3694
B,18527
C,2459


In [459]:
#generate 5 random indexes
num_index = 5
df_len = len(df)

indexes_check  = (np.random.default_rng().random((num_index,)) * df_len).round().astype('int').tolist()

In [460]:
df.iloc[indexes_check]

,npi,gender,state,credentials,specialty,opioid.prescriber,not.opioid,opioid,controlled,pop,od,od.ratio,od.group
13613,1407996044,M,IL,D.D.S.,Dentist,0,106,0,0,12882135,1705,0.013235,B
6528,1073738696,M,SC,MD,Gastroenterology,0,929,0,0,4774839,701,0.014681,B
5099,1316924798,M,AL,M.D.,Otolaryngology,0,447,0,0,4833722,723,0.014957,B
7060,1699736983,M,NY,MD,Internal Medicine,1,4340,159,266,19651127,2300,0.011704,B
20672,1366687915,F,KY,DPM,Podiatry,1,459,13,0,4395295,1077,0.024503,A


In [461]:
#test state group pairs if correct
test1(df)

Test completed


In [462]:
#subset columns
#similar dataset will be used to train the model
#sum column is generic and can be subsitution with different variations (opiods/non-opiods, etc)

df_columns = ["credentials", "specialty", "opioid.prescriber"] + opioid_col_names + ["od.group"]

df = df.loc[:,df_columns]

df.iloc[indexes_check]

,credentials,specialty,opioid.prescriber,not.opioid,opioid,controlled,od.group
13613,D.D.S.,Dentist,0,106,0,0,B
6528,MD,Gastroenterology,0,929,0,0,B
5099,M.D.,Otolaryngology,0,447,0,0,B
7060,MD,Internal Medicine,1,4340,159,266,B
20672,DPM,Podiatry,1,459,13,0,A


In [464]:
df.iloc[:2, :]

,credentials,specialty,opioid.prescriber,not.opioid,opioid,controlled,od.group
0,DDS,Dentist,1,29,0,0,C
1,MD,General Surgery,1,1304,79,35,B


In [463]:
#test_df = pd.DataFrame({"state":['NY'], "od_group":["A"]}) #incorrect group for NY state

#test1(test_df) #must raise an exception